# E-MOSAIC

引用自[E. R. Q. Fernandes, A. C. P. L. F. de Carvalho and X. Yao, "Ensemble of Classifiers Based on Multiobjective Genetic Sampling for Imbalanced Data," in IEEE Transactions on Knowledge and Data Engineering, vol. 32, no. 6, pp. 1104-1115, 1 June 2020, doi: 10.1109/TKDE.2019.2898861.]


## 导包

In [7]:
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from utils.dataset_utils import get_classes_indexes_counts
import scipy.io as sio  # 从.mat文件中读取数据集
from ucimlrepo import fetch_ucirepo




## 数据处理工具

## 数据的加载、预处理 

In [9]:
################################################################加载数据集################################################
'''
id = 12 balance_scale  
'''
# 数据集
uci_dataset = fetch_ucirepo(id=12)
#mat_data = sio.loadmat('../../data/dataset/USPS.mat') 
# 提取变量
features = uci_dataset.data.features  # 特征数据
targets = uci_dataset.data.targets  # 标签lable
# 将数据由dataframe转换成numpy格式

dataset_x = features.to_numpy()
dataset_y = targets.to_numpy()[:, 0]

# dataset_x = mat_data['X']
# dataset_y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]

# 显示数据集分布
print("特征数据:", dataset_x.shape)
print("label:", dataset_y.shape)

# One-hot encode target variable 强制将类别转换为0-1序列，0表示不是该类，1表示属于该类
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(dataset_y.reshape(-1, 1))
print("label:", y_onehot.shape)
# 统计每个类别的个数，dataset_y.max()+1是类别的个数
ttt=np.argmax(y_onehot, axis=1)
classes, counts = get_classes_indexes_counts(np.argmax(y_onehot, axis=1))
print("每种类别的数量：", counts)

###############################################################################

特征数据: (625, 4)
label: (625,)
label: (625, 3)


KeyboardInterrupt: 